# **Target Trial Emulation (TTE) with Clustering**
### CIS 3203N GROUP 3

Thesis Partner 1: Dejito, Christine Ann  
Thesis Partner 2: Roca, Zophia Maureen  
Date: March 9, 2025

---

## Instructions 
1. Look at this website: https://rpubs.com/alanyang0924/TTE
2. Extract the dummy data in the package and save it as "data_censored.csv"
2. Convert the R codes into Python Codes (use Jupyter Notebook), replicate the results using your python code.
3. Create another copy of your Python Codes, name it TTE-v2 (use Jupyter Notebook).
4. Using TTE-v2, think of a creative way on where you would integrate a clustering mechanism, understand each step carefully and decide at which step a clustering method can be implemented. Generate insights from your results.
5. Do this by pair, preferably your thesis partner.
6. Push to your github repository.

---

## What is Target Trial Emulation (TTE)?
TTE is a methodological framework in epidemiology that helps reduce biases in observational studies by mimicking the conditions of a randomized controlled trial (RCT).

### Why is this important?
- Traditional observational studies can have selection bias and confounding factors.
- TTE helps analyze causal effects (e.g., "Does Drug A reduce mortality?") in a way that is less biased.
- It constructs a hypothetical clinical trial using real-world data.

---

## Dataset
- **id**:	Unique patient ID.
- **period**:	Time period (e.g., months or years of follow-up).
- treatment	Whether the patient received treatment (1 = Yes, 0 = No).
- **x1, x2, x3, x4**:	Covariates (predictors, could be patient characteristics).
- **age**:	Patient’s age at the time of observation.
- **age_s**:	Standardized age (scaled between 0 and 1).
- **outcome**:	Event occurrence (1 = event happened, 0 = censored or no event).
- **censored**:	Whether the data is censored (1 = censored, 0 = not censored).
- **eligible**:	Whether the patient is eligible for the target trial (1 = eligible, 0 = not eligible).

---

### Import Libraries

In [12]:
import warnings
import os
import tempfile
import numpy as np
import pandas as pd
import patsy
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tools.sm_exceptions import PerfectSeparationError, ConvergenceWarning

In [13]:
# Suppress specific warnings
warnings.filterwarnings("ignore", category=PerfectSeparationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define a class to encapsulate the trial sequence and associated methods.
class TrialSequence:
    def __init__(self, estimand):
        self.estimand = estimand
        self.data = None
        self.expanded_data = None
        self.switch_weight_model = None
        self.censor_weight_model = None
        self.outcome_model_spec = None
        self.outcome_model_result = None

    def set_data(self, data, id, period, treatment, outcome, eligible):
        """Stores the observational data and variable names."""
        self.data = data.copy()
        self.id = id
        self.period = period
        self.treatment = treatment
        self.outcome = outcome
        self.eligible = eligible
        return self

    def set_switch_weight_model(self, numerator_formula, denominator_formula, model_fitter, save_path):
        """Stores specifications for the treatment switching weight model."""
        self.switch_weight_model = {
            'numerator_formula': numerator_formula,
            'denominator_formula': denominator_formula,
            'model_fitter': model_fitter,
            'save_path': save_path
        }
        return self

    def set_censor_weight_model(self, censor_event, numerator_formula, denominator_formula, pool_models, model_fitter, save_path):
        """Stores specifications for the censoring weight model."""
        self.censor_weight_model = {
            'censor_event': censor_event,
            'numerator_formula': numerator_formula,
            'denominator_formula': denominator_formula,
            'pool_models': pool_models,
            'model_fitter': model_fitter,
            'save_path': save_path
        }
        return self

    def calculate_weights(self):
        """Fit logistic regression models and calculate weights."""
        if self.switch_weight_model:
            formula = self.switch_weight_model['denominator_formula']
            y, X = patsy.dmatrices(formula, self.data, return_type="dataframe")
            model = sm.Logit(y, X).fit()
            self.data['weight'] = np.clip(1 / model.predict(X), 0, self.data['weight'].quantile(0.99) if 'weight' in self.data.columns else 1)

        if self.censor_weight_model:
            formula = self.censor_weight_model['denominator_formula']
            y, X = patsy.dmatrices(formula, self.data, return_type="dataframe")
            model = sm.Logit(y, X).fit()
            self.data['sample_weight'] = 1 / model.predict(X)

        return self

    def set_outcome_model(self, adjustment_terms=None):
        """
        Specify the outcome model. In the R code the formula included
        assigned_treatment, x2, followup_time (and its square), trial_period (and its square).
        Here we hard‐code a similar formula using patsy syntax.
        """
        self.outcome_model_spec = {
            'formula': "outcome ~ assigned_treatment + x2 + followup_time + I(followup_time**2) + trial_period + I(trial_period**2)",
            'adjustment_terms': adjustment_terms
        }
        return self

    def set_expansion_options(self, output, chunk_size):
        """
        Set options for expanding the trial data.
        For example, 'chunk_size' determines how many patients are processed at a time.
        """
        self.expansion_options = {
            'output': output,
            'chunk_size': chunk_size
        }
        return self

    def expand_trials(self):
        """
        Expand the observational dataset into a sequence of trials.
        (This is a simplified placeholder. In a real scenario, you might need to create
         multiple rows per patient with varying trial periods and follow‐up times.)
        """
        df = self.data.copy()
        # Add dummy trial-specific columns:
        df['trial_period'] = 0  # In practice, this would vary by trial
        df['followup_time'] = df[self.period]  # For example, using the period as follow-up time
        df['assigned_treatment'] = df[self.treatment]  # You might compute this differently
        self.expanded_data = df
        return self

    def load_expanded_data(self, seed=None, p_control=1.0):
        """
        For large datasets, sample control observations.
        p_control: probability to include an observation with outcome == 0.
        """
        if seed is not None:
            np.random.seed(seed)
        df = self.expanded_data.copy()
        control_mask = df[self.outcome] == 0
        sample_mask = (~control_mask) | (np.random.rand(len(df)) < p_control)
        self.expanded_data = df[sample_mask].reset_index(drop=True)
        return self

    def fit_msm(self, weight_cols, modify_weights=None):
        """
        Fit the Marginal Structural Model (MSM) using a GLM (logistic regression).
        weight_cols: list of columns that contain weights.
        modify_weights: function to winsorize or otherwise modify the weights.
        """
        formula = self.outcome_model_spec['formula']
        df = self.expanded_data.copy()
        # Modify weights if necessary:
        for col in weight_cols:
            if col in df.columns:
                if modify_weights is not None:
                    df[col] = modify_weights(df[col])
            else:
                df[col] = 1.0
        # Combine the weights (here, multiply them together)
        df['combined_weight'] = df[weight_cols].prod(axis=1)
        # Create design matrices using patsy:
        y, X = patsy.dmatrices(formula, df, return_type='dataframe')
        # Fit a logistic regression model (GLM with binomial family)
        model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=df['combined_weight'])
        result = model.fit()
        self.outcome_model_result = result
        return self

    def predict_msm(self, newdata, predict_times, type="survival"):
        """
        Generate predictions from the outcome model.
        For demonstration we use a dummy transformation of predicted probabilities.
        """
        formula = self.outcome_model_spec['formula']
        _, X_new = patsy.dmatrices(formula, newdata, return_type='dataframe')
        preds = self.outcome_model_result.predict(X_new)
        # Here we mimic survival predictions (e.g., 1 - risk) and assume a linear decay over time.
        results = []
        for t in predict_times:
            # Ensure survival probability starts at 1 and decreases over time
            surv_prob = 1 - preds * (t / max(predict_times))
            surv_prob = np.clip(surv_prob, 0, 1)  # Ensure probabilities are between 0 and 1
            results.append(surv_prob)
        survival_df = pd.DataFrame({t: results[i] for i, t in enumerate(predict_times)})
        # Optionally include follow-up time from newdata:
        if 'followup_time' in newdata.columns:
            survival_df['followup_time'] = newdata['followup_time'].values
        return survival_df

# =================== Main Workflow ===================

# 1. Setup
trial_pp = TrialSequence(estimand="PP")
trial_itt = TrialSequence(estimand="ITT")

trial_pp_dir = os.path.join(tempfile.gettempdir(), "trial_pp")
os.makedirs(trial_pp_dir, exist_ok=True)
trial_itt_dir = os.path.join(tempfile.gettempdir(), "trial_itt")
os.makedirs(trial_itt_dir, exist_ok=True)

# 2. Data Preparation
# Load the dataset from the CSV file
data_censored = pd.read_csv("data_censored.csv")
print(data_censored.head())
trial_pp.set_data(data=data_censored, id="id", period="period", treatment="treatment", outcome="outcome", eligible="eligible")
trial_itt.set_data(data=data_censored, id="id", period="period", treatment="treatment", outcome="outcome", eligible="eligible")

# 3. Weight Models and Censoring
# (Set up the models as in the R code.)
trial_pp.set_switch_weight_model(
    numerator_formula="treatment ~ age",
    denominator_formula="treatment ~ age + x1 + x3",
    model_fitter="logit",  # here we indicate that we will use a logistic model (statsmodels)
    save_path=os.path.join(trial_pp_dir, "switch_models")
)
trial_pp.set_censor_weight_model(
    censor_event="censored",
    numerator_formula="1 - censored ~ x2",
    denominator_formula="1 - censored ~ x2 + x1",
    pool_models="none",
    model_fitter="logit",
    save_path=os.path.join(trial_pp_dir, "switch_models")
)
trial_itt.set_censor_weight_model(
    censor_event="censored",
    numerator_formula="1 - censored ~ x2",
    denominator_formula="1 - censored ~ x2 + x1",
    pool_models="numerator",
    model_fitter="logit",
    save_path=os.path.join(trial_itt_dir, "switch_models")
)

# 4. Calculate Weights
trial_pp.calculate_weights()
trial_itt.calculate_weights()

# 5. Specify Outcome Model
trial_pp.set_outcome_model()
trial_itt.set_outcome_model(adjustment_terms="x2")

# 6. Expand Trials
trial_pp.set_expansion_options(output="pandas", chunk_size=500)
trial_itt.set_expansion_options(output="pandas", chunk_size=500)
trial_pp.expand_trials()
trial_itt.expand_trials()

# 7. Load or Sample from Expanded Data
trial_itt.load_expanded_data(seed=1234, p_control=0.5)

# 8. Fit Marginal Structural Model
# Here we use a lambda function to winsorize (clip) extreme weights at the 99th percentile.
trial_itt.fit_msm(
    weight_cols=["weight", "sample_weight"],
    modify_weights=lambda w: w.clip(upper=w.quantile(0.99))
)

# Print model summary (similar to show_weight_models() in R)
print(trial_itt.outcome_model_result.summary())

# 9. Inference
# For prediction, we filter the expanded data to include only a subset (e.g., trial_period == 0).
newdata = trial_itt.expanded_data[trial_itt.expanded_data['trial_period'] == 0]
predict_times = list(range(11))  # follow-up times 0 to 10
preds = trial_itt.predict_msm(newdata=newdata, predict_times=predict_times, type="survival")

# Compute mean survival probability across observations for each time point
mean_survival = preds.drop(columns=['followup_time'], errors='ignore').mean(axis=0)

# Compute confidence intervals (assuming normal approximation for demonstration)
std_survival = preds.drop(columns=['followup_time'], errors='ignore').std(axis=0)
ci_upper = mean_survival + 1.96 * std_survival / np.sqrt(len(preds))
ci_lower = mean_survival - 1.96 * std_survival / np.sqrt(len(preds))

# Plot survival difference with confidence intervals
plt.figure(figsize=(8, 6))
plt.plot(predict_times, mean_survival, 'k', label="Survival difference")  # Black solid line
plt.plot(predict_times, ci_upper, 'r--')  # Red dashed upper confidence interval
plt.plot(predict_times, ci_lower, 'r--')  # Red dashed lower confidence interval

# Labels and title
plt.xlabel("Follow up time")
plt.ylabel("Survival difference")
plt.legend()
plt.title("Survival Differences Over Time")
plt.show()

AssertionError: category must be a Warning subclass